# Multiprocessing Helper
> Things to smooth up the multiprocessing

In [ ]:
# default_exp multiproc

In [1]:
# export
from pathlib import Path
import logging
from typing import List, Dict, Callable, Any, Tuple
import random
from multiprocessing import Process

## Separate Process Magic

In [ ]:
# export
class VipaClass:
    """
    From meditation word Vipassana
    Where creating a magic function
    you can run a cell without holding the entire notebook
    ```ipython
    %%vipas
    ...do things
    ```
    """
    def __init__(self):
        self.procs = []

    def vipas(self, line, cell):
        """
        run a cell magic function, that will not hold the process
        """
        ishell = get_ipython()
        proc = Process(target=ishell.run_cell, args = (cell,), daemon=True)
        proc.start()
        self.procs.append(proc)
            
try:
    ishell = get_ipython()
    from IPython.core import magic
    vipa_class = VipaClass()
    magic.register_cell_magic(vipa_class.vipas)
    vipas = vipa_class.vipas
except:
    pass

## Read single file line by line

In [46]:
# export
class SingleFileLiner:
    """
    Text data reading line by line for multiprocessing
    """

    def __init__(self, file_path: Path, total: int = None):
        """
        filepath: Path,
            path to a textual file
        total: int
            if you set total to an integer, we'll have a fixed length interation object
            if you set total to None, we'll read the file till it's over
        """
        self.file_path = Path(file_path)
        global SingleFileLiner_fp
        SingleFileLiner_fp = open(file_path, "r")
        global text_line_num
        text_line_num = 0
        self.total = total

    def __repr__(self) -> str:
        return f"SingleFileLiner:\t{self.file_path}"

    def __next__(self) -> str:
        global SingleFileLiner_fp
        line = SingleFileLiner_fp.readline()
        global text_line_num
        text_line_num += 1
        if line == "":
            if self.total is None:
                raise StopIteration(f"SingleFileLiner file read finish")
            logging.warning(f"looping on {self.file_path}")

            self.restart()
        return line.strip()

    def __len__(self):
        if self.total is not None:
            return self.total
        else:
            raise RuntimeError(f"You have to set total for len(self)")

    def __iter__(self) -> str:
        if self.total is None:
            while True:
                try:
                    yield next(self)
                except StopIteration:
                    # end the file reading
                    break
        else:
            for i in range(self.total):
                yield next(self)

    def restart(self) -> None:
        """resetart file reader"""
        global SingleFileLiner_fp
        SingleFileLiner_fp.close()
        SingleFileLiner_fp = open(self.file_path, "r")

    def split_train_test(
            self,
            val_ratio: float = 0.1,
            remove_original: bool = False,
            logging_interval: int = 1000000) -> Tuple[Path]:
        """
        Split the text file into train and test
        :param val_ratio:
        :return:
        """
        suffix = self.file_path.suffix

        train_file_path = self.file_path.parent / \
            f"{self.file_path.stem}_train{suffix}"
        valid_file_path = self.file_path.parent / \
            f"{self.file_path.stem}_valid{suffix}"

        # delete the file if exists
        if remove_original:
            train_file_path.unlink()
            valid_file_path.unlink()

        # read and write by line
        with open(self.file_path, "r") as f, open(
            train_file_path, "w") as f_train, open(
            valid_file_path, "w") as f_valid:
            for i, line in enumerate(f):
                if i % 1000000 == 0:
                    logging.info(f"{i}\tlines processed")
                if random.random() < val_ratio:
                    f_valid.write(line)
                else:
                    f_train.write(line)
        return train_file_path, valid_file_path

In [47]:
sfl = SingleFileLiner("../README.md")
from joblib import Parallel, delayed
from time import sleep

In [48]:
def get_line(x):
#     sleep(1)
    return x

res = Parallel(backend="multiprocessing", n_jobs=6)(delayed(get_line)(i) for i in sfl)

## Read dataframe row by row

In [64]:
# export
class DataFrameRowling:
    """
    Read dataframe row by row
    """
    def __init__(self, df):
        self.df = df
        self.restart()
        
    def restart(self):
        global DataFrameRowling_df
        DataFrameRowling_df = self.df
        global DataFrameRowling_ct
        DataFrameRowling_ct = 0

    def __repr__(self) -> str:
        return f"DataFrameRowling:\t{len(self)} Rows"
        
    def __len__(self):
        return len(self.df)
    
    def __iter__(self):
        for i in range(len(self)):
            yield next(self)
            
    def __getitem__(self, idx):
        global DataFrameRowling_df
        return DataFrameRowling_df[list(DataFrameRowling_df.index)[idx]]
            
    def __next__(self):
        global DataFrameRowling_df
        global DataFrameRowling_ct
        row = DataFrameRowling_df.loc[list(DataFrameRowling_df.index)[DataFrameRowling_ct]]
        DataFrameRowling_ct+=1
        return row

In [61]:
import pandas as pd
df = pd.DataFrame({"col1":list(range(100))})

In [62]:
res = Parallel(backend="multiprocessing", n_jobs=6)(delayed(get_line)(i) for i in DataFrameRowling(df))

In [63]:
res[5]

col1    5
Name: 5, dtype: int64